# Homework \#2
Due June 30 @ 11:59pm

## Submission requirements
Upload a **single PDF file** of your IJulia notebook for this entire assigment. Clearly denote which question each section of your PDF corresponds to.

## Problem 1 -- Plant Production Planning


Prof. Smith is considering a side gig selling propogated cuttings of popular houseplants. The propogation process involves putting the plant cuttings (or seeds) in a medium (often water) and waiting for roots to grow. The rooted cuttings can then be planted and/or sold. The propogation process is, unfortunately, highly variable. Prof. Smith has categorized the results of the process into five "classes" of cuttings: "sickly," "poor," "fair," "good," and "excellent." Two different methods can be used to propogate cuttings. On average, method 1 costs \\$0.50 per cutting and method 2 costs \\$0.40 per cutting. The qualities of rooted cuttings produced by each method are shown in the following table (in percent of total cuttings):

|Quality|Method 1|Method 2|
|:---|:---|:---|
|sickly|30|20|
|poor|30|20|
|fair|20|25|
|good|15|20|
|excellent|5|15|

Prof. Smith has a rooting enzyme that she can apply to propogated cuttings in an attempt to improve the outcome. It costs \$0.25 per cutting to use the enzyme. The results of the attempted improvements can be seen in the table below (in percent of total cuttings). (For example, rooting enzyme applied to good cuttings would produce excellent cuttings 50\% of the time, but would not change the quality the remaining 50\% of the time.)

|Quality|sickly|poor|fair|good|excellent|
|:---|:---|:---|:---|:---|:----|
|sickly|30|25|15|20|10|
|poor|-|30|30|20|20|
|fair|-|-|40|30|30|
|good|-|-|-|50|50|

Prof. Smith has sufficient space at home to propogate at most 200 cuttings per month. Her anticipated monthly demands are 10 excellent cuttings, 20 good cuttings, 30 fair cuttings, and 30 poor cuttings.

(a) Assuming only one application of rooting enzyme is allowed, model the propogation process and determine Prof. Smith's minimum cost strategy for meeting monthly demand.

In [47]:
using JuMP, Clp, NamedArrays

### DATA ###
method = [:1, :2]
quality = [:sickly, :poor, :fair, :good, :excellent]
method_matrix = [30 20
                 30 20
                 20 25
                 15 20
                 5  15]
method_NA = NamedArray(method_matrix, (quality, method), ("quality", "method"))
m1_cost = 0.5
m2_cost = 0.4
enzyme_matrix = [30 25 15 20 10
                 0  30 30 20 20
                 0  0  40 30 30
                 0  0  0  50 50
                 0  0  0  0  100] # added excellence row to make NamedArray available
enzyme_NA = NamedArray(enzyme_matrix, (quality,quality), ("quality", "quality"))
enzyme_cost = 0.25
max_cutting = 200
quality_demand = Dict(zip(quality, [0, 30, 30, 20, 10]));


### MODEL ###

m = Model(Clp.Optimizer)
@variable(m, x1[quality] >= 0)
@variable(m, x2[quality] >= 0)

# aggregated cost from each method
@expression(m, m_tot_cost[i in quality], sum( (x1[i] * m1_cost) + (x2[i] * m2_cost)) )
# 
@expression(m, enzyme_tot_cost, sum( (x1[i] + x2[i]) * enzyme_cost for i in quality) )
@expression(m, tot_cost, sum(m_tot_cost) + enzyme_tot_cost)

@objective(m, Min, tot_cost)
# @objective(m, Min, cost_per_haul*sum( x[i,j]*dist[i,j] for i in sites, j in crimelords ) ) 

# meet quantity requirement: sum of cuttings from method 1 + method 2 should be equal to 200.
@constraint(m, quantity_req, sum(x1) + sum(x2) == max_cutting)
# meet monthly demand requirement
# the sum of quality from each method should exceed demand of each cutting.
@constraint(m, monthly_req[j in quality], sum( (x1[i] + x2[i]) * enzyme_NA[i,j] / 100 for i in quality) >=  quality_demand[j] )

optimize!(m)

println("Total Cost: ", objective_value(m))

Total Cost: 130.0
Coin0506I Presolve 5 (-1) rows, 5 (-5) columns and 19 (-21) elements
Clp0006I 0  Obj 0 Primal inf 474.71287 (5)
Clp0006I 1  Obj 130
Clp0000I Optimal - objective value 130
Coin0511I After Postsolve, objective 130, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 130 - 1 iterations time 0.002, Presolve 0.00


(b) The model in part (a) was for a single month of operation. To be profitable, Prof. Smith knows she needs to plan at least 3 months in advance. She anticipates demand for the following three months to be:

|Month|poor|fair|good|excellent|
|:---|:---|:---|:---|:---|
|1|30|30|20|10|
|2|40|25|15|15|
|3|20|10|15|30|

It costs Prof. Smith \\$0.10 to store cuttings produced in one month for a future month. As in part (a), only one application of rooting enzyme is allowed for each cutting. Rooting enzyme cannot be applied to stored cuttings, regardless of whether or not it was applied in a prior month. Modify your model from part (a) to plan for the next three month's of production at a minimum cost. 

## Problem 2 -- Aging like a fine...coffee?

Coffee Co is considering expanding their roast options to include a special line of aged beans. This is a limited-time-only offering, so Coffee Co only has a total of $n=8$ bags of beans dedicated to this. The current age of each bag of beans is $a_i$ (in months) and is listed in the table below. Each month, we  update the current age of bag $i$ as $T_i = a_i + t$ for $t \in \{0,1,2,3,4,5,6,7\}$. The flavor of the beans deepens with age. The flavor profile can be described as a function of the age of the beans: $f_i(T_i) = b_iT_i^3 - c_iT_i$ for some known coefficients $b_i$ and $c_i$. All the relevant data are summarized in the following table: 

| Bag | $a$| $b$|$c$|
|----|---|----|---|
| 1|20|2|10|
|2|5|3|15|
|3|4|5|20|
|4|20|3|5|
|5|10|1|25|
|6|15|4|15|
|7|9|4|10|
|8|12|5|20|

Formulate and solve a linear program that will allow Coffee Co to select one bag of beans to sell each month for the next 8 months that will maximize the overall flavor profile of beans sold. Bags can only be sold once (once a bag is sold, it cannot be sold again in a future month).

In [46]:
### DATA ###
bags = [:1, :2, :3, :4, :5, :6, :7, :8]
a = [20 5 4 20 10 15 9 12]
current_age = Dict(zip(bag, a))
coef_b = Dict(zip(bag, [2 3 5 3 1 4 4 5]))
coef_c = Dict(zip(bag, [10 15 20 5 25 15 10 20]))
months = [0, 1, 2, 3, 4, 5, 6, 7]

function calc_flavor(i, t)
    result = coef_b[i] * (a[i] + t)^3 - coef_c[i] * (a[i] + t)
    result
end

temp_array = Int64[]

m = Model(Clp.Optimizer)
#while length(bags) > 0
@variable(m, x[bags] >= 0)

for bag in bags
    push!(temp_array, calc_flavor(bag, 0)) 
end

flavors = Dict(zip(bag, temp_array))
println(flavors)        

Dict(5 => 750, 4 => 23900, 6 => 13275, 7 => 2826, 2 => 300, 8 => 8400, 3 => 240, 1 => 15800)


## Problem 3 -- We've got the power!

Madison Gas \& Electric is planning power generation for the next 24 hours and would like to consult an optimization expert for advice. The figure shown below represents the power distribution network that connects power generating points with power consuming points. Each node represents a neighborhood in the greater Madison area.

![](hw2_network.jpg)


Note that these arcs are all undirected, meaning power can flow either direction along each edge in the network. Neighborhoods 1, 4, and 7 and power generating points with the following generator supply and unit costs:

|Generator|1|4|7|
|:---|:--|:--|:--|
|Capacity (in thousands of KWH)|90|50|85|
|Unit cost (\\$/thousand KWH)|14|13.5|21|

Neighborhoods 2, 5, and 6 are power consuming nodes with demands of 30,000, 50,000, and 60,000 KWH, respectively. There is no limit to how much power can be sent along the transmission lines connecting the neighborhoods. It costs \\$10 per 1000 KWH to use a transmission line. 

Build and solve this problem to minimize the cost of meeting the power demand. Formulate this as a min-cost network flow problem (linear program). Make sure you use directed arcs in your formulation (_hint:_ you will have more arcs in your formulation than are shown in the figure).

In [1]:
# Min-Cost Network Flow Problem (5)

using JuMP, Clp, NamedArrays

### DATA ###
nodes = [:1,:2,:3,:4,:5,:6,:7]
source = [:1, :4, :7]
sink = [:2, :5, :6]
unit_cost = Dict(zip(source, [14 13.5 21]))

arcs = [(:1,:2), (:1,:5), (:1,:6),(:2,:1), (:2,:3), (:2,:7), (:3,:2), (:3,:4), (:3,:5), 
    (:4,:2), (:4,:3), (:4,:5), (:4,:6), (:5,:3), (:5,:4), (:6,:7), (:7,:2), (:7,:5), (:7,:6)]

straight_arcs = [(:1,:2), (:4,:5), (:7,:6)]

flow_arcs = [(:1,:5), (:1,:6), (:4,:2), (:4,:6), (:7,:2), (:7,:5)]

g1_supply = [(:1,:2), (:1,:5), (:1,:6)]
g4_supply = [(:4,:2), (:4,:5), (:4,:6)]
g7_supply = [(:7,:2), (:7,:5), (:7,:6)]
g2_demand = [(:1,:2), (:4,:2), (:7,:2)]
g5_demand = [(:1,:5), (:4,:5), (:7,:5)]
g6_demand = [(:1,:6), (:4,:6), (:7,:6)]

### MODEL ###

m = Model(Clp.Optimizer)

@variable(m, x[arcs] >= 0)

@expression(m, straight_cost, sum(x[arc] * unit_cost[arc[1]] for arc in straight_arcs))
@expression(m, flow_cost[arc in flow_arcs], (unit_cost[arc[1]] + 10) * x[arc])

@objective(m, Min, sum(straight_cost) + sum(flow_cost))

# supply requirements
@constraint(m, supply_req_1, sum(x[s] for s in g1_supply) <= 90000)
@constraint(m, supply_req_4, sum(x[s] for s in g4_supply) <= 50000)
@constraint(m, supply_req_7, sum(x[s] for s in g7_supply) <= 85000)

# demand requirements
@constraint(m, demand_req_2, sum(x[d] for d in g2_demand) == 30000)
@constraint(m, demand_req_5, sum(x[d] for d in g2_demand) == 50000)
@constraint(m, demand_req_6, sum(x[d] for d in g2_demand) == 60000)

# balance flow in ~ out
@constraint(m, x[(:2,:3)] + x[(:4,:3)] + x[(:5,:3)] == x[(:3,:2)] + x[(:3,:4)] + x[(:3,:5)])


optimize!(m)
println("Total cost will be \$", objective_value(m))

Total cost will be $840000.0
Coin0507I Presolve determined that the problem was infeasible with tolerance of 1e-08
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj 0 Primal inf 140000 (3)
Clp0001I Primal infeasible - objective value 840000
Clp0032I PrimalInfeasible objective 840000 - 1 iterations time 0.012
